In [22]:
import pypdf
import re

fp = "data/sub/O6_1A_Umweltgerechte_und_biodiversitaetsfoerdernde_Bewirtschaftung_(UBB)_2023_04.pdf"

# Create a PDF object
reader = pypdf.PdfReader(fp)

reader.pages[0].extract_text()

FileNotFoundError: [Errno 2] No such file or directory: 'data/sub/O6_1A_Umweltgerechte_und_biodiversitaetsfoerdernde_Bewirtschaftung_(UBB)_2023_04.pdf'

# Parsing PDF into Nodes

In [ ]:
import pdfplumber

# Code to extract chapters and subchapters from pdf
def extract_chapters(pdf_path):
    data = {'chapters': []}

    with pdfplumber.open(pdf_path) as pdf:
        pages = range(len(pdf.pages))
        for i in pages:
            page = pdf.pages[i]
            text = page.extract_text()
            lines = text.split('\n')
            current_chapter = None
            current_subchapter = None

            for j, line in enumerate(lines):
                if line.isupper():  # Identifying chapters
                    current_chapter = line.strip()
                    data['chapters'].append(
                        {'title': current_chapter, 'subchapters': []})
                elif current_chapter and line.strip() and not line.isnumeric():  # Identifying subchapters
                    current_subchapter = line.strip()
                    data['chapters'][-1]['subchapters'].append(
                        {'title': current_subchapter, 'text': []})
                elif current_chapter and current_subchapter and line.strip():  # Adding text to subchapter
                    data['chapters'][-1]['subchapters'][-1]['text'].append(
                        line.strip())
                    
                else:
                    print(f'Warning: No chapter or subchapter found for page {i} line {j}')
                    print(f'Could add to previous subchapter {current_subchapter}')
                    print(line, "\n")

    return data


structured_pdf = extract_chapters(fp)

# dump as json with encoding utf-8
import json
with open('structure.json', 'w', encoding='utf-8') as f:
    json.dump(structured_pdf, f, ensure_ascii=False, indent=4)


Could add to previous subchapter None
8.3 Blühpflanzen, Heil- und Gewürzpflanzen ............................................................ 20 

Could add to previous subchapter None
8.4 Förderungswürdige Ackerkulturen ...................................................................... 20 

Could add to previous subchapter None
8.5 Über 7 % hinausgehende Biodiversitätsflächen auf Ackerflächen ...................... 22 

Could add to previous subchapter None
8.6 Mindestens 1 Biodiversitätsfläche je 3,00 ha Ackerfläche................................... 22 

Could add to previous subchapter None
8.7 Acker-Biodiversitätsflächen und Ackerzahl ......................................................... 22 

Could add to previous subchapter None
8.8 Über 7 % hinausgehende Biodiversitätsflächen auf Grünlandflächen ................. 23 

Could add to previous subchapter None
8.9 Mindestens 1 Biodiversitätsfläche je 3,00 ha Grünlandfläche ............................. 23 

Could add to previ

In [ ]:
import pdfplumber


def extract_chapters(pdf_path):
    data = {'chapters': []}
    current_chapter = None
    current_subchapter = None

    with pdfplumber.open(pdf_path) as pdf:
        for i, page in enumerate(pdf.pages):
            text = page.extract_text()
            lines = text.split('\n')

            for j, line in enumerate(lines):
                if line.isupper():  # Identifying chapters
                    current_chapter = line.strip()
                    data['chapters'].append(
                        {'title': current_chapter, 'subchapters': []})
                    current_subchapter = None  # Reset subchapter when a new chapter is found
                elif current_chapter and line.strip() and not line.isnumeric():  # Identifying subchapters
                    current_subchapter = line.strip()
                    data['chapters'][-1]['subchapters'].append(
                        {'title': current_subchapter, 'text': []})
                elif current_chapter and current_subchapter and line.strip():  # Adding text to subchapter
                    data['chapters'][-1]['subchapters'][-1]['text'].append(
                        line.strip())

    return data


structured_pdf = extract_chapters(fp)


In [ ]:
def extract_title(structured_pdf):
    title = [chapter for chapter in structured_pdf['chapters']
             if chapter['title'] == 'ÖPUL 2023'][0]['subchapters'][0]['title']

    return title


extract_title(structured_pdf)


'Umweltgerechte und biodiversitätsfördernde Bewirtschaftung'

In [ ]:
# import json
# def extract_chapters(structured_pdf):
#     content_list_raw  = title = [chapter for chapter in structured_pdf['chapters']
#                        if chapter['title'] == 'INHALT'][0]['subchapters'][1:]
    
#     content_list = [item['title'] for item in content_list_raw]

#     cleaned_data = [re.sub(
#         r'(\b\w+\b)\s+\.{3,}\s+(\d+)$', r'\1 \2', item) for item in content_list]
    
#     hierarchical_data = {}

#     for item in cleaned_data:
#         line = item.rstrip('.')
#         parts = line.split(' ', 1)
#         number, title = parts[0], parts[1]
#         number_parts = number.split('.')
#         current_dict = hierarchical_data
#         for num in number_parts:
#             if num not in current_dict:
#                 current_dict[num] = {}
#             current_dict = current_dict[num]

#         stripped_title = title.strip()
#         current_dict['title'] = " ".join(stripped_title.split(" ")[0:-1])
#         current_dict['page'] = stripped_title.split(" ")[-1]

#     return hierarchical_data


# hierarchical_data = extract_chapters(structured_pdf)

# with open('hierarchy.json', 'w', encoding='utf-8') as f:
#     json.dump(hierarchical_data, f, ensure_ascii=False, indent=4)


In [ ]:
# link some chapter to structure

chapter = structured_pdf['chapters'][20]

index = chapter['title'].split(' ')[0]
title = chapter['title'].split(' ')[1:]
text = " ".join([line['title'] for line in chapter['subchapters']])

# find index and get structure
print(f'Index: {index}')
print(f'Title: {title}')
text




Index: 6.1.4.2
Title: ['PFLEGE-/NUTZUNGSAUFLAGEN']


'Die Biodiversitätsfläche muss mindestens 1x jedes zweite Jahr gemäht oder gehäckselt werden. Mähen/Häckseln ist maximal 2x pro Jahr erlaubt. Das Mähgut darf von der Fläche verbracht und genutzt werden. Auf 75 % der Biodiversitätsflächen ist je Kalenderjahr Mähen/Häckseln frühestens am 1. August erlaubt, auf den anderen 25 % ist dies ohne zeitliche Einschränkungen möglich. Für die Berechnung der DIV-Fläche, welche frühestens am 1. August (75 %) gehäckselt/gemäht werden darf, müssen alle DIV-Flächen unabhängig von der möglichen zusätzlichen Codierung (BAW, AG, NAT, EBW) berücksichtigt werden. Beispiel: Ein Betrieb mit 2,5 ha Grünbrache + DIV und 1,5 ha Grünbrache + DIV + NAT (Projektbestätigung Häckseln ab 01.07.): Gesamte DIV-Fläche: 4 ha, davon 75 % = 3 ha frühestens am 1. August, 1 ha vor dem 1. August (25 %) Folgendes ist erlaubt: • 1,5 ha Grünbrache + DIV+ NAT: Häckseln ab 1. Juli gemäß Projektbestätigung • 2,5 ha Grünbrache + DIV: Häckseln/Mähen ab 1. August Nachdem die Projektbes

In [ ]:
from llama_index.schema import Document, NodeRelationship, RelatedNodeInfo

doc_title = extract_title(structured_pdf)

header_stack = [doc_title]
pdf_docs = []
prev_index = None


chapters = structured_pdf['chapters']
for chapter in chapters:
    index = chapter['title'].split(' ')[0]
    index_list = index.split('.')
    text = " ".join([line['title'] for line in chapter['subchapters']]).strip()

    # If the index is numeric, update header stack
    if all([item.isnumeric() for item in index_list]):
        title = " ".join(chapter['title'].split(' ')[1:]).strip()

        header_stack_len = len(header_stack) - 1
        index_len = len(index_list)

        # If new subheader present add it
        if header_stack_len < index_len:
            header_stack.append(title)
        # If going back in hierarchy remove elements to be shorter than index
        elif header_stack_len > index_len:
            [header_stack.pop() for _ in range(header_stack_len - index_len + 1)]
            header_stack.append(title)
        else:
            header_stack.pop()
            header_stack.append(title)
        
        # Add document to list
        pdf_docs.append(
            Document(
                text=text,
                metadata={
                    'Filename': "filename",
                    "Content Type": "text",
                    'header_stack': "/".join(header_stack)
                }
            )
        )

    # Else just link to parent element
    else:
        pdf_docs.append(
            Document(
                text=text,
                metadata={
                    'Filename': "filename",
                    "Content Type": "text",
                    'header_stack': "/".join(header_stack) + f"/{chapter['title']}"
                }
            )
        )


[Document(id_='93ddcc23-0c1c-4991-81aa-97cf36ca2005', embedding=None, metadata={'Filename': 'filename', 'Content Type': 'text', 'header_stack': 'Umweltgerechte und biodiversitätsfördernde Bewirtschaftung/K-Ö'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, hash='033f35c7439fea39ff7c371757931cab2ecfadd7ef5c05349dbfb1715c04be52', text='', start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'),
 Document(id_='2bf3c183-2acc-4dba-865f-88f5d7f513c0', embedding=None, metadata={'Filename': 'filename', 'Content Type': 'text', 'header_stack': 'Umweltgerechte und biodiversitätsfördernde Bewirtschaftung/ÖPUL 2023'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, hash='8a6b3ba81e5075f02bb9263d453ab9916a0369376510a0ff580a3c8c9141e090', text='Umweltgerechte und biodiversitätsfördernde Bewirtschaftung STAND April 2023', start_char_idx=None, e

In [ ]:
["x" for i in range(1)]

['x']

In [3]:
# import camelot
# tables = camelot.read_pdf(
#     'tmp/O6_1A_Umweltgerechte_und_biodiversitaetsfoerdernde_Bewirtschaftung_(UBB)_2023_04.pdf')

OSError: Ghostscript is not installed. You can install it using the instructions here: https://camelot-py.readthedocs.io/en/master/user/install-deps.html

In [2]:
from ctypes.util import find_library

find_library("gs")